In [2]:
# =============================================================================
# CÉLULA 1: IMPORTAÇÕES E CONFIGURAÇÕES INICIAIS
# =============================================================================
"""
Importação das bibliotecas necessárias para o projeto de Machine Learning
para predição de causas de óbito no dataset DATASUS.
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import zipfile
import io
import os
from IPython.display import display

# Machine Learning
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (
    classification_report, confusion_matrix, accuracy_score, 
    precision_recall_fscore_support, roc_auc_score, roc_curve, auc
)
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

# Modelos
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# Configurações
import warnings
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
pd.set_option('display.max_columns', None)

### OBJETIVO DO PROJETO:
Prever a causa básica de óbito (CAUSABAS) utilizando características demográficas
e circunstâncias do óbito do dataset DATASUS (DO24OPEN).

### RELEVÂNCIA:
Este projeto é relevante para:
- Análise epidemiológica: Identificar padrões nas causas de morte
- Saúde pública: Auxiliar na alocação de recursos e políticas de prevenção
- Predição: Classificar óbitos em categorias de causa para análise automática

### TÉCNICAS UTILIZADAS:
1. Regressão Logística: Modelo linear interpretável para classificação
2. Random Forest: Ensemble com múltiplas árvores para capturar não-linearidades
3. SVM (Support Vector Machine): Modelo não-linear para classificação multiclasse (TÉCNICA EXTRA)
4. XGBoost: Gradient boosting avançado para melhor performance

### METODOLOGIA:
- Pré-processamento: Limpeza, encoding, normalização
- Balanceamento: SMOTE para lidar com desbalanceamento de classes
- Validação: Cross-validation e divisão treino/teste
- Otimização: Grid Search para hiperparâmetros
- Avaliação: Múltiplas métricas (Acurácia, Precisão, Recall, F1-Score)

In [8]:
# =============================================================================
# CÉLULA 3: FUNÇÃO PARA CARREGAMENTO DE DADOS
# =============================================================================
def carregar_csv_do_zip_remoto(url_zip, nome_csv = None, cache_local = True):
    """
    Carrega um arquivo CSV de um ZIP remoto com cache local
    
    Parâmetros:
    -----------
    url_zip : str
        URL do arquivo ZIP
    nome_csv : str, optional
        Nome do arquivo CSV dentro do ZIP
    cache_local : bool, default=True
        Se True, salva o CSV localmente
    
    Retorna:
    --------
    pd.DataFrame
        DataFrame com os dados carregados
    """
    
    print("=" * 80)
    print("CARREGANDO DADOS DE FORMA REMOTA")
    print("=" * 80)
    
    # Verificar se arquivo local existe
    if cache_local and nome_csv and os.path.exists(nome_csv):
        print(f"\nArquivo local encontrado: {nome_csv}")
        print("  Carregando do cache local...")
        try:
            df = pd.read_csv(nome_csv, delimiter=';', low_memory=False, encoding='latin1')
            print(f"Dataset carregado com sucesso!")
            print(f"Dimensões: {df.shape[0]} linhas x {df.shape[1]} colunas")
            return df
        except Exception as e:
            print(f"Erro ao carregar arquivo local: {e}")
            print("Tentando baixar novamente...")
    
    print(f"\nBaixando arquivo ZIP remoto...")
    print(f"   URL: {url_zip}")
    
    try:
        response = requests.get(url_zip, timeout=300)
        response.raise_for_status()
        print(f"Download concluído ({len(response.content) / (1024**2):.2f} MB)")
    except Exception as e:
        print(f"Erro ao baixar: {e}")
        raise
    
    print(f"\nExtraindo arquivo ZIP...")
    try:
        zip_file = zipfile.ZipFile(io.BytesIO(response.content))
        print(f"Arquivos no ZIP: {zip_file.namelist()}")
    except Exception as e:
        print(f" Erro ao extrair ZIP: {e}")
        raise
    
    if nome_csv is None:
        csv_files = [f for f in zip_file.namelist() if f.endswith('.csv')]
        if not csv_files:
            raise ValueError("Nenhum arquivo CSV encontrado no ZIP")
        nome_csv = csv_files[0]
        print(f"Arquivo CSV encontrado: {nome_csv}")
    
    print(f"\nLendo arquivo CSV...")
    try:
        with zip_file.open(nome_csv) as csv_file:
            df = pd.read_csv(csv_file, delimiter=';', low_memory=False, encoding='latin1')
        print(f"✓ Dataset carregado com sucesso!")
        print(f"  Dimensões: {df.shape[0]} linhas × {df.shape[1]} colunas")
    except Exception as e:
        print(f"✗ Erro ao ler CSV: {e}")
        raise
    
    # Salvar localmente (cache)
    if cache_local:
        print(f"\nSalvando arquivo localmente para futuras execuções...")
        try:
            df.to_csv(nome_csv, index=False, sep=';', encoding='latin1')
            print(f" Arquivo salvo: {nome_csv}")
        except Exception as e:
            print(f" Aviso: Não foi possível salvar localmente: {e}")
    
    print("\n" + "=" * 80)
    return df

In [6]:
# =============================================================================
# CÉLULA 4: CARREGAMENTO DO DATASET
# =============================================================================
"""
Carregamento do dataset DATASUS com dados de óbitos.
URL oficial do DATASUS para dados de 2024.
"""

url_zip = "https://s3.sa-east-1.amazonaws.com/ckan.saude.gov.br/SIM/csv/DO24OPEN_csv.zip"

# Carregar dados
df = carregar_csv_do_zip_remoto(
    url_zip = url_zip,
    nome_csv = 'DO24OPEN.csv',
    cache_local = True
)

# Exibir informações
print("\nDados carregados!")
print(f"Shape: {df.shape}")
print(f"\nPrimeiras linhas:")
print(df.head())

CARREGANDO DADOS DE FORMA REMOTA

Arquivo local encontrado: DO24OPEN.csv
  Carregando do cache local...
Dataset carregado com sucesso!
Dimensões: 1426346 linhas x 86 colunas

Dados carregados!
Shape: (1426346, 86)

Primeiras linhas:
   contador  ORIGEM  TIPOBITO   DTOBITO  HORAOBITO  NATURAL  CODMUNNATU  \
0         1       1         2  19042024     2301.0    824.0    240490.0   
1         2       1         2  18012024     1030.0    824.0    240940.0   
2         3       1         2  22012024      700.0    824.0    241250.0   
3         4       1         2  24032024     1220.0    824.0    241050.0   
4         5       1         2  12012024      800.0    824.0    240320.0   

       DTNASC  IDADE  SEXO  RACACOR  ESTCIV  ESC  ESC2010  SERIESCFAL  \
0  24121964.0    459     1      1.0     1.0  1.0      0.0         NaN   
1  29041958.0    465     2      1.0     2.0  5.0      5.0         NaN   
2  23031939.0    484     2      4.0     2.0  1.0      0.0         NaN   
3  10061944.0    479    

In [7]:
# =============================================================================
# CÉLULA 5: EXPLORAÇÃO INICIAL DOS DADOS
# =============================================================================
"""
Análise exploratória inicial do dataset para entender estrutura e tipos de dados.
"""

print("\n[1] EXPLORAÇÃO INICIAL DOS DADOS...")

try:
    df = pd.read_csv('DO24OPEN.csv', delimiter=';', low_memory=False, encoding='latin1')
    print(f"Dataset carregado com sucesso!")
    print(f"  Dimensões: {df.shape[0]} linhas × {df.shape[1]} colunas")
except Exception as e:
    print(f"Erro ao carregar: {e}")
    exit()

print("\nColunas disponíveis:")
print(df.columns.tolist())

print("\nTipos de dados:")
print(df.dtypes)


[1] EXPLORAÇÃO INICIAL DOS DADOS...
Dataset carregado com sucesso!
  Dimensões: 1426346 linhas × 86 colunas

Colunas disponíveis:
['contador', 'ORIGEM', 'TIPOBITO', 'DTOBITO', 'HORAOBITO', 'NATURAL', 'CODMUNNATU', 'DTNASC', 'IDADE', 'SEXO', 'RACACOR', 'ESTCIV', 'ESC', 'ESC2010', 'SERIESCFAL', 'OCUP', 'CODMUNRES', 'LOCOCOR', 'CODESTAB', 'CODMUNOCOR', 'IDADEMAE', 'ESCMAE', 'ESCMAE2010', 'SERIESCMAE', 'OCUPMAE', 'QTDFILVIVO', 'QTDFILMORT', 'GRAVIDEZ', 'SEMAGESTAC', 'GESTACAO', 'PARTO', 'OBITOPARTO', 'PESO', 'TPMORTEOCO', 'OBITOGRAV', 'OBITOPUERP', 'ASSISTMED', 'EXAME', 'CIRURGIA', 'NECROPSIA', 'LINHAA', 'LINHAB', 'LINHAC', 'LINHAD', 'LINHAII', 'CAUSABAS', 'CB_PRE', 'COMUNSVOIM', 'DTATESTADO', 'CIRCOBITO', 'ACIDTRAB', 'FONTE', 'NUMEROLOTE', 'DTINVESTIG', 'DTCADASTRO', 'ATESTANTE', 'STCODIFICA', 'CODIFICADO', 'VERSAOSIST', 'VERSAOSCB', 'FONTEINV', 'DTRECEBIM', 'ATESTADO', 'DTRECORIGA', 'OPOR_DO', 'CAUSAMAT', 'ESCMAEAGR1', 'ESCFALAGR1', 'STDOEPIDEM', 'STDONOVA', 'DIFDATA', 'NUDIASOBCO', 'DT

In [9]:
# =============================================================================
# CÉLULA 6: SELEÇÃO DE VARIÁVEIS RELEVANTES
# =============================================================================
"""
Seleção das variáveis mais relevantes para o modelo de Machine Learning.
Variáveis escolhidas com base em relevância epidemiológica e disponibilidade.
"""

# Seleção de variáveis relevantes para predição
# Variáveis demográficas e características da morte
variaveis_selecionadas = [
    'IDADE_CALCULADA',  # Idade
    'SEXO',              # Sexo (1=Masculino, 2=Feminino)
    'RACACOR',           # Raça/Cor
    'ESC2010',           # Escolaridade
    'LOCOCOR',           # Local de ocorrência
    'CIRCOBITO',         # Circunstância do óbito
    'CAUSABAS'           # Causa básica (alvo)
]

# Verificar quais colunas existem
colunas_existentes = [col for col in variaveis_selecionadas if col in df.columns]
print(f"Variáveis selecionadas: {colunas_existentes}")

# Criar dataset com variáveis selecionadas
df_ml = df[colunas_existentes].copy()

print(f"\nDataset para ML: {df_ml.shape}")
print(f"Valores ausentes:\n{df_ml.isnull().sum()}")

Variáveis selecionadas: ['SEXO', 'RACACOR', 'ESC2010', 'LOCOCOR', 'CIRCOBITO', 'CAUSABAS']

Dataset para ML: (1426346, 6)
Valores ausentes:
SEXO               0
RACACOR        19445
ESC2010        91225
LOCOCOR            0
CIRCOBITO    1283156
CAUSABAS           0
dtype: int64


In [10]:
# =============================================================================
# CÉLULA 7: ANÁLISE E REMOÇÃO DE OUTLIERS
# =============================================================================
"""
Detecção e remoção de outliers usando método IQR (Interquartile Range).
Outliers podem distorcer o treinamento dos modelos.
"""

print("\n[2] ANÁLISE E REMOÇÃO DE OUTLIERS...")

# Selecionar colunas numéricas para análise de outliers
numeric_columns = df.select_dtypes(include=[np.number]).columns

# Calcular IQR para cada coluna
iqr_values = {}
for col in numeric_columns:
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    iqr_values[col] = iqr

# Identificar outliers
outliers = {}
for col in numeric_columns:
    lower_bound = df[col].quantile(0.25) - 1.5 * iqr_values[col]
    upper_bound = df[col].quantile(0.75) + 1.5 * iqr_values[col]
    outliers[col] = df[(df[col] < lower_bound) | (df[col] > upper_bound)]

# Remover outliers
for col in numeric_columns:
    df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]

print(f"\nOutliers removidos: {outliers['IDADE'].shape[0]} linhas")
print(f"Dimensões do dataset após remoção de outliers: {df.shape[0]} linhas x {df.shape[1]} colunas")



[2] ANÁLISE E REMOÇÃO DE OUTLIERS...

Outliers removidos: 54422 linhas
Dimensões do dataset após remoção de outliers: 0 linhas x 86 colunas


In [11]:
# =============================================================================
# CÉLULA 8: TRATAMENTO DA VARIÁVEL ALVO (CAUSABAS)
# =============================================================================
"""
Agrupamento das causas de óbito menos frequentes em categoria "Outras"
para reduzir dimensionalidade e melhorar performance dos modelos.
"""

print("\n[4] TRATAMENTO DA VARIÁVEL ALVO (CAUSABAS)...")

# Verificar distribuição da causa básica
print(f"\nDistribuição de causas (top 10):")
print(df_ml['CAUSABAS'].value_counts().head(10))

# Agrupar causas menos frequentes em "Outras"
# Manter apenas as 10 principais causas
top_causas = df_ml['CAUSABAS'].value_counts().head(10).index
df_ml['CAUSABAS_AGRUPADA'] = df_ml['CAUSABAS'].apply(
    lambda x: x if x in top_causas else 'Outras'
)

print(f"\nDistribuição após agrupamento:")
print(df_ml['CAUSABAS_AGRUPADA'].value_counts())


[4] TRATAMENTO DA VARIÁVEL ALVO (CAUSABAS)...

Distribuição de causas (top 10):
CAUSABAS
I219    84189
J189    58934
R99     48773
I64     32454
I10     31055
C349    28372
N390    27756
G309    26281
A419    23612
E149    22485
Name: count, dtype: int64

Distribuição após agrupamento:
CAUSABAS_AGRUPADA
Outras    1042435
I219        84189
J189        58934
R99         48773
I64         32454
I10         31055
C349        28372
N390        27756
G309        26281
A419        23612
E149        22485
Name: count, dtype: int64


In [12]:
# =============================================================================
# CÉLULA 9: LIMPEZA DE DADOS
# =============================================================================
"""
Remoção de valores ausentes e preparação para encoding.
"""
# Remover linhas com valores ausentes
df_ml = df_ml.dropna()
print(f"Dataset após remover NaN: {df_ml.shape}")

Dataset após remover NaN: (133658, 7)


In [13]:
# =============================================================================
# CÉLULA 10: CODIFICAÇÃO DE VARIÁVEIS CATEGÓRICAS
# =============================================================================
"""
Transformação de variáveis categóricas para numéricas usando LabelEncoder.
Essencial para que os modelos de Machine Learning possam processar os dados.
"""

print("\n[5] CODIFICAÇÃO DE VARIÁVEIS CATEGÓRICAS...")

# Separar variáveis numéricas e categóricas
X = df_ml.drop(['CAUSABAS', 'CAUSABAS_AGRUPADA'], axis=1)
y = df_ml['CAUSABAS_AGRUPADA']

# Dicionário para armazenar encoders
label_encoders = {}

# Codificar variáveis categóricas
for col in X.columns:
    if X[col].dtype == 'object':
        le = LabelEncoder()
        X[col] = le.fit_transform(X[col].astype(str))
        label_encoders[col] = le
        print(f"  ✓ {col} codificado")

        # Codificar variável alvo
le_target = LabelEncoder()
y_encoded = le_target.fit_transform(y)

print(f"\nVariáveis de entrada (X): {X.shape}")
print(f"Variável alvo (y): {y_encoded.shape}")
print(f"Classes: {le_target.classes_}")


[5] CODIFICAÇÃO DE VARIÁVEIS CATEGÓRICAS...

Variáveis de entrada (X): (133658, 5)
Variável alvo (y): (133658,)
Classes: ['C349' 'I10' 'I219' 'I64' 'J189' 'Outras' 'R99']


In [ ]:
# =============================================================================
# CÉLULA 11: DIVISÃO DOS DADOS EM TREINO E TESTE
# =============================================================================
"""
Divisão estratificada dos dados em conjuntos de treino (80%) e teste (20%).
Estratificação garante distribuição similar das classes em ambos os conjuntos.
"""

print("\n[6] DIVISÃO DOS DADOS...")

X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size = 0.2, random_state = 42, stratify = y_encoded
)

print(f"Conjunto de treino: {X_train.shape}")
print(f"Conjunto de teste: {X_test.shape}")
print(f"\nDistribuição no treino:\n{pd.Series(y_train).value_counts()}")
print(f"\nDistribuição no teste:\n{pd.Series(y_test).value_counts()}")

# Aplicação de SMOTE (Synthetic Minority Over-sampling Technique)
# para balancear as classes no conjunto de treino, evitando bias do modelo.
smote = SMOTE(random_state = 42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

print(f"Treino após SMOTE: {X_train_balanced.shape}")
print(f"Distribuição após SMOTE:\n{pd.Series(y_train_balanced).value_counts()}")

In [ ]:
# =============================================================================
# CÉLULA 13: NORMALIZAÇÃO DOS DADOS
# =============================================================================
"""
Aplicação de StandardScaler para normalizar as features.
Essencial para modelos sensíveis à escala como Regressão Logística e SVM.
"""

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_balanced)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# =============================================================================
# CÉLULA 14: SELEÇÃO DE ATRIBUTOS (FEATURE SELECTION)
# =============================================================================
"""
Seleção das features mais relevantes usando SelectKBest com f_classif.
Reduz dimensionalidade e melhora performance dos modelos.
"""

print("\n[14] SELEÇÃO DE ATRIBUTOS...")

# Usar SelectKBest para selecionar as melhores features
selector = SelectKBest(f_classif, k='all')
X_train_selected = selector.fit_transform(X_train_balanced, y_train_balanced)
X_test_selected = selector.transform(X_test)

print(f"\nFeatures selecionadas: {selected_features.tolist()}")
print(f"Dimensão do dataset selecionado: {X_selected.shape[1]}")

# Treinar os modelos com as features selecionadas
models_selected = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Random Forest': RandomForestClassifier(),
    'SVM': SVC(),
    'XGBoost': XGBClassifier()
}

for model_name, model in models_selected.items():
    model.fit(X_selected, y_train)
    y_pred = model.predict(X_selected)
    print(f"\nModelo: {model_name}")
    print(classification_report(y_train, y_pred))
    print(confusion_matrix(y_train, y_pred))
    print("Acurácia: ", accuracy_score(y_train, y_pred))
    print("Precisão: ", precision_score(y_train, y_pred, average='macro'))
    print("Recall: ", recall_score(y_train, y_pred, average='macro'))
    print("F1-Score: ", f1_score(y_train, y_pred, average='macro'))

# Salvar os modelos treinados
for model_name, model in models_selected.items():
    joblib.dump(model, f"model_{model_name.lower()}.joblib")

print("\nModelos treinados e salvo com sucesso!")

In [ ]:
# =============================================================================
# CÉLULA 15: TÉCNICA 1 - REGRESSÃO LOGÍSTICA
# =============================================================================
"""
Treinamento de Regressão Logística com Grid Search e validação cruzada.
Modelo baseline interpretável para classificação multiclasse.
"""

param_grid_lr = {
    'C': [0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'saga']
}

grid_lr = GridSearchCV(
    estimator = LogisticRegression(max_iter=1000),
    param_grid = param_grid_lr,
    cv = 5,
    scoring = 'accuracy',
    n_jobs = -1,
    verbose = 1
)

grid_lr.fit(X_train, y_train)

print("\nMelhores parâmetros para Regressão Logística:")
print(grid_lr.best_params_)

y_pred_lr = grid_lr.predict(X_test)

print("\nRelatório de classificação para Regressão Logística:")
print(classification_report(y_test, y_pred_lr))

print("\nMatriz de confusão para Regressão Logística:")
print(confusion_matrix(y_test, y_pred_lr))

print("\nAcurácia para Regressão Logística:")
print(accuracy_score(y_test, y_pred_lr))

print("\nPrecisão para Regressão Logística:")
print(precision_score(y_test, y_pred_lr, average = 'macro'))

print("\nRecall para Regressão Logística:")
print(recall_score(y_test, y_pred_lr, average = 'macro'))

print("\nF1 - Score para Regressão Logística:")
print(f1_score(y_test, y_pred_lr, average = 'macro'))

In [ ]:
# Visualizar matriz de confusão
plt.figure(figsize=(10, 8))
sns.heatmap(cm_lr, annot=True, fmt='d', cmap='Blues', 
            xticklabels=le_target.classes_, yticklabels=le_target.classes_)
plt.title('Matriz de Confusão - Regressão Logística')
plt.ylabel('Verdadeiro')
plt.xlabel('Predito')
plt.tight_layout()
plt.savefig('matriz_confusao_lr.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# =============================================================================
# CÉLULA 16: TÉCNICA 2 - RANDOM FOREST
# =============================================================================
"""
Treinamento de Random Forest com Grid Search e validação cruzada.
Ensemble robusto que captura relações não-lineares.
"""

# Grid Search para otimizar hiperparâmetros
param_grid_rf = {
    'n_estimators': [50, 100],
    'max_depth': [10, 15, 20],
    'min_samples_split': [5, 10]
}

rf_base = RandomForestClassifier(random_state=42, n_jobs=-1)
grid_search_rf = GridSearchCV(rf_base, param_grid_rf, cv=5, scoring='f1_weighted', n_jobs=-1)
grid_search_rf.fit(X_train_selected, y_train_balanced)

print(f"Melhores parâmetros (RF): {grid_search_rf.best_params_}")
print(f"Melhor score CV (RF): {grid_search_rf.best_score_:.4f}")

rf_model = grid_search_rf.best_estimator_

# Validação cruzada
cv_scores_rf = cross_val_score(rf_model, X_train_selected, y_train_balanced, cv=5, scoring='f1_weighted')
print(f"Cross-validation scores (RF): {cv_scores_rf}")
print(f"Média CV (RF): {cv_scores_rf.mean():.4f} (+/- {cv_scores_rf.std():.4f})")

# Predições
y_pred_rf = rf_model.predict(X_test_selected)
y_pred_proba_rf = rf_model.predict_proba(X_test_selected)

# Avaliação
print("\nRESULTADOS - RANDOM FOREST:")
print(f"Acurácia: {accuracy_score(y_test, y_pred_rf):.4f}")
print(f"Precisão (média): {precision_score(y_test, y_pred_rf, average='weighted', zero_division=0):.4f}")
print(f"Recall (média): {recall_score(y_test, y_pred_rf, average='weighted', zero_division=0):.4f}")
print(f"F1-Score (média): {f1_score(y_test, y_pred_rf, average='weighted', zero_division=0):.4f}")

print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred_rf, target_names=le_target.classes_, zero_division=0))

print("\nMatriz de Confusão:")
cm_rf = confusion_matrix(y_test, y_pred_rf)
print(cm_rf)

# Visualizar matriz de confusão
plt.figure(figsize=(10, 8))
sns.heatmap(cm_rf, annot=True, fmt='d', cmap='Greens',
            xticklabels=le_target.classes_, yticklabels=le_target.classes_)
plt.title('Matriz de Confusão - Random Forest')
plt.ylabel('Verdadeiro')
plt.xlabel('Predito')
plt.tight_layout()
plt.savefig('matriz_confusao_rf.png', dpi=300, bbox_inches='tight')
plt.show()

# Importância das features
feature_importance_rf = pd.DataFrame({
    'feature': X.columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(10, 6))
plt.barh(feature_importance_rf['feature'], feature_importance_rf['importance'])
plt.xlabel('Importância')
plt.title('Importância das Features - Random Forest')
plt.tight_layout()
plt.savefig('importancia_features_rf.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# =============================================================================
# CÉLULA 17: TÉCNICA 3 - SUPPORT VECTOR MACHINE (SVM) - TÉCNICA EXTRA
# =============================================================================
"""
Treinamento de Support Vector Machine com Grid Search e validação cruzada.
Técnica não vista em sala para pontuação extra.
"""

# Grid Search para SVM
param_grid_svm = {
    'C': [0.1, 1, 10],
    'kernel': ['rbf', 'poly'],
    'gamma': ['scale', 'auto']
}

svm_base = SVC(random_state=42, probability=True)
grid_search_svm = GridSearchCV(svm_base, param_grid_svm, cv=5, scoring='f1_weighted', n_jobs=-1)
grid_search_svm.fit(X_train_scaled_selected, y_train_balanced)

print(f"Melhores parâmetros (SVM): {grid_search_svm.best_params_}")
print(f"Melhor score CV (SVM): {grid_search_svm.best_score_:.4f}")
svm_model = grid_search_svm.best_estimator_

# Validação cruzada
cv_scores_svm = cross_val_score(svm_model, X_train_scaled_selected, y_train_balanced, cv=5, scoring='f1_weighted')
print(f"Cross-validation scores (SVM): {cv_scores_svm}")
print(f"Média CV (SVM): {cv_scores_svm.mean():.4f} (+/- {cv_scores_svm.std():.4f})")

# Predições
y_pred_svm = svm_model.predict(X_test_scaled_selected)
y_pred_proba_svm = svm_model.predict_proba(X_test_scaled_selected)

# Avaliação
print("\nRESULTADOS - SUPPORT VECTOR MACHINE:")
print(f"Acurácia: {accuracy_score(y_test, y_pred_svm):.4f}")
print(f"Precisão (média): {precision_score(y_test, y_pred_svm, average='weighted', zero_division=0):.4f}")
print(f"Recall (média): {recall_score(y_test, y_pred_svm, average='weighted', zero_division=0):.4f}")
print(f"F1-Score (média): {f1_score(y_test, y_pred_svm, average='weighted', zero_division=0):.4f}")

print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred_svm, target_names=le_target.classes_, zero_division=0))

print("\nMatriz de Confusão:")
cm_svm = confusion_matrix(y_test, y_pred_svm)
print(cm_svm)

# Visualizar matriz de confusão
plt.figure(figsize=(10, 8))
sns.heatmap(cm_svm, annot=True, fmt='d', cmap='Purples',
            xticklabels=le_target.classes_, yticklabels=le_target.classes_)
plt.title('Matriz de Confusão - SVM')
plt.ylabel('Verdadeiro')
plt.xlabel('Predito')
plt.tight_layout()
plt.savefig('matriz_confusao_svm.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# =============================================================================
# CÉLULA 18: TÉCNICA 4 - XGBOOST
# =============================================================================
"""
Treinamento de XGBoost com Grid Search e validação cruzada.
Gradient boosting avançado para melhor performance.
"""

print("\n" + "=" * 80)
print("TÉCNICA 4: XGBOOST")
print("=" * 80)

print("\nTreinando modelo de XGBoost com Grid Search...")

# Grid Search para XGBoost
param_grid_xgb = {
    'n_estimators': [50, 100],
    'max_depth': [5, 6, 7],
    'learning_rate': [0.05, 0.1]
}

xgb_base = XGBClassifier(random_state=42, n_jobs=-1, verbosity=0)
grid_search_xgb = GridSearchCV(xgb_base, param_grid_xgb, cv=5, scoring='f1_weighted', n_jobs=-1)
grid_search_xgb.fit(X_train_selected, y_train_balanced)

print("\nMelhores parâmetros encontrados:")
print(grid_search_xgb.best_params_)

# Treinar o modelo final com os melhores parâmetros
xgb_final = grid_search_xgb.best_estimator_
xgb_final.fit(X_train_selected, y_train_balanced)

# Validação cruzada
cv_scores_xgb = cross_val_score(xgb_model, X_train_selected, y_train_balanced, cv=5, scoring='f1_weighted')
print(f"Cross-validation scores (XGB): {cv_scores_xgb}")
print(f"Média CV (XGB): {cv_scores_xgb.mean():.4f} (+/- {cv_scores_xgb.std():.4f})")

# Avaliar o modelo final
y_pred_xgb = xgb_final.predict(X_test_selected)
y_pred_proba_xgb = xgb_model.predict_proba(X_test_selected)

print("\nResultados do modelo final:")
print(classification_report(y_test_balanced, y_pred_xgb))

# Salvar o modelo final
joblib.dump(xgb_final, 'xgboost_final.joblib')

In [ ]:
# Avaliação
print("\nRESULTADOS - XGBOOST:")
print(f"Acurácia: {accuracy_score(y_test, y_pred_xgb):.4f}")
print(f"Precisão (média): {precision_score(y_test, y_pred_xgb, average='weighted', zero_division=0):.4f}")
print(f"Recall (média): {recall_score(y_test, y_pred_xgb, average='weighted', zero_division=0):.4f}")
print(f"F1-Score (média): {f1_score(y_test, y_pred_xgb, average='weighted', zero_division=0):.4f}")

In [ ]:
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred_xgb, target_names=le_target.classes_, zero_division=0))

In [ ]:
print("\nMatriz de Confusão:")
cm_xgb = confusion_matrix(y_test, y_pred_xgb)
print(cm_xgb)

In [ ]:
# Visualizar matriz de confusão
plt.figure(figsize = (10, 8))
sns.heatmap(cm_xgb, annot = True, fmt = 'd', cmap = 'Oranges',
            xticklabels = le_target.classes_, yticklabels = le_target.classes_)
plt.title('Matriz de Confusão - XGBoost')
plt.ylabel('Verdadeiro')
plt.xlabel('Predito')
plt.tight_layout()
plt.savefig('matriz_confusao_xgb.png', dpi = 300, bbox_inches = 'tight')
plt.show()

In [ ]:
# Importância das features
feature_importance_xgb = pd.DataFrame({
    'feature': X.columns,
    'importance': xgb_model.feature_importances_
}).sort_values('importance', ascending=False)

In [ ]:
print("\nImportância das Features (XGBoost):")
print(feature_importance_xgb)

In [ ]:
plt.figure(figsize=(10, 6))
plt.barh(feature_importance_xgb['feature'], feature_importance_xgb['importance'])
plt.xlabel('Importância')
plt.title('Importância das Features - XGBoost')
plt.tight_layout()
plt.savefig('importancia_features_xgb.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# =============================================================================
# CÉLULA 19: COMPARAÇÃO DOS MODELOS
# =============================================================================
"""
Comparação visual e numérica entre todos os modelos treinados.
Identificação do melhor modelo com base nas métricas de avaliação.
"""

In [ ]:
print("\n" + "=" * 80)
print("COMPARAÇÃO DOS MODELOS")
print("=" * 80)

In [ ]:
# Criar tabela de comparação
comparacao = pd.DataFrame({
    'Modelo': ['Regressão Logística', 'Random Forest', 'SVM', 'XGBoost'],
    'Acurácia': [
        accuracy_score(y_test, y_pred_lr),
        accuracy_score(y_test, y_pred_rf),
        accuracy_score(y_test, y_pred_svm),
        accuracy_score(y_test, y_pred_xgb)
    ],
    'Precisão': [
        precision_score(y_test, y_pred_lr, average='weighted', zero_division=0),
        precision_score(y_test, y_pred_rf, average='weighted', zero_division=0),
        precision_score(y_test, y_pred_svm, average='weighted', zero_division=0),
        precision_score(y_test, y_pred_xgb, average='weighted', zero_division=0)
    ],
    'Recall': [
        recall_score(y_test, y_pred_lr, average='weighted', zero_division=0),
        recall_score(y_test, y_pred_rf, average='weighted', zero_division=0),
        recall_score(y_test, y_pred_svm, average='weighted', zero_division=0),
        recall_score(y_test, y_pred_xgb, average='weighted', zero_division=0)
    ],
    'F1-Score': [
        f1_score(y_test, y_pred_lr, average='weighted', zero_division=0),
        f1_score(y_test, y_pred_rf, average='weighted', zero_division=0),
        f1_score(y_test, y_pred_svm, average='weighted', zero_division=0),
        f1_score(y_test, y_pred_xgb, average='weighted', zero_division=0)
    ]
})

In [ ]:
print("\n" + comparacao.to_string(index=False))

In [ ]:
# Visualizar comparação
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
metricas = ['Acurácia', 'Precisão', 'Recall', 'F1-Score']
for idx, metrica in enumerate(metricas):
    ax = axes[idx // 2, idx % 2]
    valores = comparacao[metrica].values
    modelos = comparacao['Modelo'].values
    bars = ax.bar(range(len(modelos)), valores, color=['#1f77b4', '#2ca02c', '#ff7f0e', '#d62728'], edgecolor='black', linewidth=1.5)
    ax.set_ylabel(metrica, fontsize=11, fontweight='bold')
    ax.set_title(f'{metrica} por Modelo', fontsize=12, fontweight='bold')
    ax.set_ylim([0, 1])
    ax.set_xticks(range(len(modelos)))
    ax.set_xticklabels(modelos, rotation=45, ha='right')
    ax.grid(axis='y', alpha=0.3)

    # Adicionar valores nas barras
    for i, v in enumerate(valores):
        ax.text(i, v + 0.02, f'{v:.3f}', ha='center', va='bottom', fontweight='bold', fontsize=10)

In [ ]:
plt.tight_layout()
plt.savefig('comparacao_modelos.png', dpi=300, bbox_inches='tight')
print("\n✓ Gráfico salvo: comparacao_modelos.png")
plt.show()

In [ ]:
# =============================================================================
# CÉLULA 20: ANÁLISE DETALHADA DOS RESULTADOS
# =============================================================================
"""
Análise crítica dos resultados obtidos, identificando o melhor modelo,
discutindo limitações e propondo melhorias futuras.
"""

print("\n" + "=" * 80)
print("ANÁLISE DETALHADA DOS RESULTADOS")
print("=" * 80)

melhor_modelo_idx = comparacao['F1-Score'].idxmax()
melhor_modelo = comparacao.loc[melhor_modelo_idx, 'Modelo']
melhor_f1 = comparacao.loc[melhor_modelo_idx, 'F1-Score']

print(f"\nMELHOR MODELO: {melhor_modelo}")
print(f"  F1-Score: {melhor_f1:.4f}")
print(f"  Acurácia: {comparacao.loc[melhor_modelo_idx, 'Acurácia']:.4f}")
print(f"  Precisão: {comparacao.loc[melhor_modelo_idx, 'Precisão']:.4f}")
print(f"  Recall: {comparacao.loc[melhor_modelo_idx, 'Recall']:.4f}")

print("\n" + "=" * 80)
print("PONTOS FORTES E LIMITAÇÕES DOS MODELOS")
print("=" * 80)

print("\n1. REGRESSÃO LOGÍSTICA:")
print("   Pontos Fortes:")
print("   - Modelo simples e interpretável")
print("   - Rápido para treinar")
print("   - Bom para dados linearmente separáveis")
print("   Limitações:")
print("   - Pode não capturar relações não-lineares complexas")
print("   - Sensível a outliers")

print("\n2. RANDOM FOREST:")
print("   Pontos Fortes:")
print("   - Captura relações não-lineares")
print("   - Robusto a outliers")
print("   - Fornece importância das features")
print("   Limitações:")
print("   - Pode sofrer overfitting com dados pequenos")
print("   - Menos interpretável que regressão logística")

print("\n3. SUPPORT VECTOR MACHINE (SVM):")
print("   Pontos Fortes:")
print("   - Excelente para classificação multiclasse")
print("   - Funciona bem com dados de alta dimensionalidade")
print("   - Kernels não-lineares capturam padrões complexos")
print("   Limitações:")
print("   - Computacionalmente caro para datasets grandes")
print("   - Menos interpretável")
print("   - Sensível ao scaling dos dados")

print("\n4. XGBOOST:")
print("   Pontos Fortes:")
print("   - Melhor performance em muitos casos")
print("   - Regularização integrada reduz overfitting")
print("   - Fornece importância das features")
print("   Limitações:")
print("   - Mais complexo de tunar")
print("   - Requer mais dados para performance ótima")

In [ ]:


print("\n1. COLETA DE DADOS:")
print("   - Aumentar o volume de dados para melhor generalização")
print("   - Balancear melhor as classes de causa de óbito")

In [ ]:
print("\n2. ENGENHARIA DE FEATURES:")
print("   - Criar features derivadas (ex: idade em grupos)")
print("   - Explorar interações entre variáveis")
print("   - Aplicar técnicas de redução de dimensionalidade (PCA)")

In [ ]:
print("\n3. OTIMIZAÇÃO DE MODELOS:")
print("   - Explorar mais combinações de hiperparâmetros")
print("   - Usar ensemble de múltiplos modelos (Stacking, Voting)")
print("   - Implementar técnicas de regularização adicional")

In [ ]:
print("\n4. VALIDAÇÃO:")
print("   - Usar estratificação em todas as divisões")
print("   - Aplicar validação temporal se dados forem sequenciais")
print("   - Testar em dados completamente separados")

In [ ]:
print(f"""
Este projeto demonstrou a aplicação de 4 técnicas de Machine Learning para
predição de causas de óbito no dataset DATASUS:

1. Regressão Logística: Modelo baseline interpretável
2. Random Forest: Ensemble robusto com boa performance
3. Support Vector Machine: Técnica não vista em sala (ponto extra)
4. XGBoost: Gradient boosting avançado

METODOLOGIA APLICADA:
✓ Pré-processamento: Limpeza, remoção de outliers, encoding
✓ Seleção de atributos: SelectKBest com f_classif
✓ Balanceamento: SMOTE para lidar com desbalanceamento
✓ Normalização: StandardScaler para modelos sensíveis
✓ Otimização: Grid Search com validação cruzada (5-fold)
✓ Avaliação: Múltiplas métricas (Acurácia, Precisão, Recall, F1-Score)

RESULTADO PRINCIPAL:
O modelo {melhor_modelo} apresentou o melhor F1-Score ({melhor_f1:.4f}),
indicando melhor balance entre precisão e recall para esta tarefa.

APLICAÇÕES PRÁTICAS:
- Análise epidemiológica automatizada
- Suporte a decisões em saúde pública
- Identificação de padrões em mortalidade
- Alocação de recursos de prevenção
""")

print("=" * 80)
print("ANÁLISE CONCLUÍDA COM SUCESSO!")
print("=" * 80)